In [1]:
%matplotlib qt

In [2]:
import time
import numpy as np
import cv2

In [13]:
def angle_cos(p0, p1, p2):
    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

def reorder_vertices(vertices):
    x_min = np.min(vertices[:,0])
    x_max = np.max(vertices[:,0])
    y_min = np.min(vertices[:,1])
    y_max = np.max(vertices[:,1])
    new_vertices = np.zeros((4,2), dtype=np.int32)
    new_vertices[0,0], new_vertices[0,1] = x_min, y_min
    new_vertices[1,0], new_vertices[1,1] = x_max, y_min
    new_vertices[2,0], new_vertices[2,1] = x_max, y_max
    new_vertices[3,0], new_vertices[3,1] = x_min, y_max
    return new_vertices

def find_all_contours(img):
    # get image area
    img_area = np.prod(img.shape[:2])
    
    img = cv2.GaussianBlur(img, (5, 5), 0)
    buttons = []
    for gray in cv2.split(img):
        for thrs in range(0, 255, 26):
            if thrs == 0:
                bin = cv2.Canny(gray, 0, 50, apertureSize=5)
                bin = cv2.dilate(bin, None)
            else:
                _retval, bin = cv2.threshold(gray, thrs, 255, cv2.THRESH_BINARY)
            contours, _hierarchy = cv2.findContours(bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                cnt_len = cv2.arcLength(cnt, True)
                cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
                if len(cnt) == 4 and cv2.isContourConvex(cnt):
                    cnt_area = cv2.contourArea(cnt)
                    if cnt_area/img_area > 0.02 and cnt_area/img_area < 0.10:
                        cnt = cnt.reshape(-1, 2)
                        max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                        if max_cos < 0.1:
                            cnt = reorder_vertices(cnt)
                            buttons.append(cnt)
    return buttons

def remove_duplicate_contours(img, buttons):
    # remove buttons based on proximity and intersection
    threshold = 50
    pos_y = [buttons[i][0][1] for i in range(len(buttons))]
    idx_y = np.argsort(pos_y)
    new_idx = []
    tmp_idx = []

    for i in range(len(idx_y)):
        if len(tmp_idx) == 0:
            tmp_idx.append(idx_y[i])
        if abs(pos_y[idx_y[i]] - pos_y[tmp_idx[-1]]) < threshold:
            if tmp_idx[-1] != idx_y[i]:
                tmp_idx.append(idx_y[i])
        else:
            pos_x = [buttons[j][0][0] for j in tmp_idx]
            idx_x = np.argsort(pos_x)
            tmp2_idx = []

            for j in range(len(idx_x)):
                if len(tmp2_idx) == 0:
                    tmp2_idx.append(tmp_idx[idx_x[j]])
                if abs(buttons[tmp_idx[idx_x[j]]][0][0] - buttons[tmp2_idx[-1]][0][0] ) > threshold:
                    tmp2_idx.append(tmp_idx[idx_x[j]])
            new_idx.extend(tmp2_idx)
            tmp_idx = []
    pos_x = [buttons[j][0][0] for j in tmp_idx]
    idx_x = np.argsort(pos_x)
    tmp2_idx = []

    for j in range(len(idx_x)):
        if len(tmp2_idx) == 0:
            tmp2_idx.append(tmp_idx[idx_x[j]])
        if abs(buttons[tmp_idx[idx_x[j]]][0][0] - buttons[tmp2_idx[-1]][0][0] ) > threshold:
            tmp2_idx.append(tmp_idx[idx_x[j]])
    new_idx.extend(tmp2_idx)
    new_buttons = [buttons[i] for i in new_idx]
    
    return new_buttons

def get_dominant_color(img):
    px = cv2.resize(img, (1,1))[0,0]
    return px

def validate_buttons(img, buttons, verbose=1):
    # Get button statistics
    img_area = np.prod(img.shape[:2])
    area = []
    for i in range(len(buttons)):
        area.append(cv2.contourArea(buttons[i]))
    median_area = np.median(area)
    
    valid_buttons = []
    for i in range(len(buttons)):
        if verbose == 1:
            print('.',end='')
        
        # crop to button
        x,y,w,h = cv2.boundingRect(buttons[i])
        crop = img[y:y+h,x:x+w]
        cv2.imwrite('output/btn_%d.png' % i, crop) # save picture

        # check if button has normal size
        if abs(area[i] - median_area) <= median_area * 0.25:
            # check amount of white space in button
            crop_middle = crop[int(crop.shape[0]/4):int(crop.shape[0]/4*3),int(crop.shape[1]/4):int(crop.shape[1]/4*3)]
            common_pix = get_dominant_color(crop_middle)
            num_background = np.count_nonzero((crop_middle == common_pix).all(axis = 2))
            num_pixels = np.prod(crop_middle.shape[:2])
            if verbose == 2:
                print('%d: %0.2f' % (i, num_background / num_pixels))
            if num_background / num_pixels > 0.99:
                continue

        # button is valid, add to list
        valid_buttons.append(buttons[i])
    if verbose == 1:
        print('')
    return valid_buttons

def remove_contained_buttons(buttons):
    new_buttons = []
    for i in range(len(buttons)):
        btn_contained = False
        for j in range(len(buttons)):
            if i != j and (buttons[i][0,0] >= buttons[j][0,0]) and (buttons[i][0,1] >= buttons[j][0,1]) and (buttons[i][2,0] <= buttons[j][2,0]) and (buttons[i][2,1] <= buttons[j][2,1]):
                btn_contained = True
                break
        if btn_contained == False:
            new_buttons.append(buttons[i])
    return new_buttons

def detect_buttons(img):
    # find contours
    buttons = find_all_contours(img)
    buttons = remove_duplicate_contours(img, buttons)
    buttons = validate_buttons(img, buttons)
    buttons = remove_contained_buttons(buttons)
    return buttons

In [18]:
# DETECT BUTTONS

# Read image
# fn = 'data/c5_home.png'
# fn = 'data/c5_things.png'
# fn = 'data/c5_toys.png'
# fn = 'data/c5_short-words.png'
# fn = 'data/c5_abc.png'
fn = 'data/c5_scene1.png'
img = cv2.imread(fn)
# img = cv2.resize(img, (0,0), fx=0.8, fy=0.8)

# detect buttons
start = time.time()
buttons = detect_buttons(img)
end = time.time()
print("[INFO] button detection took {:.6f} seconds".format(end - start))
print('num buttons: ', len(buttons))

# plot detection
plot = img.copy()
cv2.drawContours( plot, buttons, -1, (0, 255, 0), 3 )
cv2.imshow('buttons', plot)
ch = cv2.waitKey()
cv2.destroyAllWindows()

# save output
cv2.imwrite('output/output.png', plot)

.................
[INFO] button detection took 0.933815 seconds
num buttons:  11


True